In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

import os
from dotenv import load_dotenv
from constant import *
load_dotenv()


# Initialize ChatOpenAI with GPT-4o-mini
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    max_tokens=512,
    verbose=False,
    openai_api_key = os.getenv('OPENAI_API_KEY')
)

# Load your dictionary file (table & column descriptions)
def load_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

# Prepare prompt templates
system_template = """
You are a PostgreSQL expert for a jewelry store.
Given the database schema:

{schema}

And the meaning of each table and column:

{dictionary}

Generate ONLY a valid PostgreSQL SELECT SQL query to answer the user question.
Do not add explanations or markdown, just output the SQL.
If you are not sure about the question, just output 'I am not sure on this question.'
"""

human_template = "{user_question}"

def build_prompt(schema: str, dictionary: str, user_question: str):
    system_msg = SystemMessagePromptTemplate.from_template(system_template)
    human_msg = HumanMessagePromptTemplate.from_template(human_template)
    prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
    return prompt.format_prompt(schema=schema, dictionary=dictionary, user_question=user_question).to_messages()

def generate_sql(user_question, schema, dictionary):
    messages = build_prompt(schema, dictionary, user_question)
    response = llm(messages)
    return response.content

schema_str = SCHEMA
dictionary_str = load_file(DICT_DIR)

print("💎 Jewelry Inventory SQL Generator (type 'exit' to quit)")

while True:
    user_question = input("\nAsk your question: ").strip()
    if user_question.lower() in {"exit", "quit"}:
        break

    print("\nUser's question:\n")
    print(user_question)

    sql = generate_sql(user_question, schema_str, dictionary_str)
    print("\nGenerated SQL query:\n" + sql)
    break

💎 Jewelry Inventory SQL Generator (type 'exit' to quit)

User's question:

what is the total stock weight I ahve?

Generated SQL query:
SELECT SUM(stk_weight) AS total_stock_weight FROM STOCK;


In [28]:
import os
from dotenv import load_dotenv
import pandas as pd
import psycopg2
import logging
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from constant import *

# ====== LOAD ENVIRONMENT ======
load_dotenv()

# ====== LOGGING SETUP ======
log_file = "db_query.log"
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ====== SIMPLE DATABASE CLASS ======
class Database: 
    # def __init__(self, database_url: str):
    def __init__(self, host: str,port: str,database: str,user: str,password: str):
        # self.database_url = database_url
        # self.conn = psycopg2.connect(self.database_url)
        self.conn = psycopg2.connect(host=host,port=port,database=database,user=user,password=password)
        # self.cursor = self.conn.cursor()
        self.cursor = None
        self.schema = 'konghin'
        self.conn.autocommit = False

        
    def select_raw(self, query: str, params: tuple = None, js: bool = False):
        """
        Executes a raw SQL query with optional parameters.

        Args:
            query (str): The SQL query to execute.
            params (tuple, optional): Parameters to safely substitute into the query.
            js (bool, optional): If True, returns the result as a JSON string.

        Returns:
            pd.DataFrame or str: DataFrame of results or JSON if js=True.
        """
        try:
            # self.refresh_connection()
            
            if self.cursor is None or self.cursor.closed:
                self.cursor = self.conn.cursor()
            
            self.cursor.execute(query, params)
            results = self.cursor.fetchall()
            colnames = [desc[0] for desc in self.cursor.description]

            df = pd.DataFrame(results, columns=colnames)

            logging.info(f"Successfully executed query: {query}")

            if js:
                return df.to_json(orient='records', date_format='iso')
            return df

        except psycopg2.Error as e:
            logging.error(f"Database error: {e.pgcode} - {e.pgerror}")
            return None
        except Exception as e:
            logging.error(f"Query: {query.as_string(self.conn)}")
            logging.error(f"Unexpected error: {e}")
            return None

    def __del__(self):
        if hasattr(self, 'cursor') and self.cursor:
            self.cursor.close()
        if hasattr(self, 'conn') and self.conn:
            self.conn.close()

# ====== LOAD DICTIONARY FILE ======
def load_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

# ====== LLM SETUP ======
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    max_tokens=512,
    verbose=False,
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

# ====== PROMPT TEMPLATES ======
system_template = """
You are a PostgreSQL expert for a jewelry store.
Given the database schema:

{schema}

And the meaning of each table and column:

{dictionary}

Generate ONLY a valid PostgreSQL SELECT SQL query to answer the user question.
pls add schema name before table name. Example: SELECT * FROM konghin.stock.
Do NOT use double quotes ("") around schema, table names, or column names.
Use lowercase for schema and table names.
Do not add explanations or markdown, just output the SQL.
If you are not sure about the question, just output 'I am not sure on this question.'
"""
human_template = "{user_question}"

def build_prompt(schema: str, dictionary: str, user_question: str):
    system_msg = SystemMessagePromptTemplate.from_template(system_template)
    human_msg = HumanMessagePromptTemplate.from_template(human_template)
    prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
    return prompt.format_prompt(schema=schema, dictionary=dictionary, user_question=user_question).to_messages()

def generate_sql(user_question, schema, dictionary):
    messages = build_prompt(schema, dictionary, user_question)
    response = llm(messages)
    return response.content

# ====== FUNCTION TO ANSWER QUESTIONS WITH DATA ======
def answer_with_data(df: pd.DataFrame, user_question: str):
    """
    Pass the DataFrame back to the LLM to answer the original question.
    """
    data_str = df.to_csv(index=False)
    analysis_prompt = f"""
    You are a data analyst for a jewelry store. 
    The user asked: "{user_question}"
    Here is the query result:
    {data_str}

    Based on this data, provide a concise and accurate answer.
    If user ask in mandarin, pls reply in mandarin.
    """
    messages = [HumanMessagePromptTemplate.from_template(analysis_prompt).format()]
    response = llm(messages)
    return response.content

# ====== MAIN PROGRAM ======
if __name__ == "__main__":
    schema_str = SCHEMA
    dictionary_str = load_file(DICT_DIR)

    db = Database(
        host=os.environ["HOST"],
        port=os.environ["PORT"],
        database=os.environ["DATABASE"],
        user=os.environ["USER"],
        password=os.environ["PASSWORD"]
        )

    print("💎 Jewelry Inventory Question Answering (type 'exit' to quit)")
    while True:
        user_question = input("\nAsk your question: ").strip()
        if user_question.lower() in {"exit", "quit"}:
            break

        print("\nUser's question:\n", user_question)

        # Step 1: Generate SQL
        sql_query = generate_sql(user_question, schema_str, dictionary_str)
        print("\nGenerated SQL query:\n", sql_query)

        if sql_query.strip().lower() == "i am not sure on this question.":
            print("LLM could not generate a confident SQL query.")
            continue

        # Step 2: Run SQL on Database
        df = db.select_raw(sql_query)
        if df is None or df.empty:
            print("No results found or query error.")
            continue

        # Step 3: Pass data back to LLM for final answer
        final_answer = answer_with_data(df, user_question)
        print("\nFinal Answer:\n", final_answer)
        break


💎 Jewelry Inventory Question Answering (type 'exit' to quit)

User's question:
 请列出所有客户（Customer）的姓名和联系方式，以及他们最近一次的预订（Booking）日期和该预订的总价（book_price）。如果客户没有预订，则显示预订信息为空。

Generated SQL query:
 SELECT c.cust_name, c.cust_phone_number, c.cust_email_address, b.book_date, b.book_price
FROM konghin.customer c
LEFT JOIN konghin.booking b ON c.cust_id = b.book_cust_id;

Final Answer:
 根据查询结果，以下是所有客户的姓名和联系方式，以及他们最近一次的预订日期和该预订的总价：

| 姓名         | 联系电话      | 电子邮件                   | 最近预订日期  | 总价     |
|--------------|---------------|----------------------------|---------------|----------|
| Kuan         | 012           | 213@gmail.com              | 2025-02-23    | 1000.00  |
| test         | 554-1234      | john.doe@example.com       | 2024-12-12    | 0.00     |
| test 2      | 555-8765      | alice.johnson@example.com  | 2024-12-12    | 80.00    |
| test 3      | 555-4321      | bob.brown@example.com      | 空            | 空       |
| test 1      | 555-5678      | jane.smith@example.com     | 空 

## Intent Classifier

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os

import yaml
from src.shared.constant import *

INTENT_SYSTEM_PROMPT = """
  You are an intent classification engine for a jewelry store chatbot.
  
  Classify the user's question into exactly ONE of the following intents:
  
  - general_knowledge
  - database_query
  - store_info
  - unclear
  
  Rules:
    - If question asks for numbers, counts, sales, stock, transactions → database_query
    - If question asks about return, exchange, warranty, policy → store_policy
    - If question asks about the store background, history, brand → store_info
    - If question is general world knowledge → general_knowledge
    - If unsure → unclear
  
  Respond with ONLY the intent label.
"""


intent_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=10,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

def classify_intent(user_question: str) -> str:
    messages = [
        SystemMessage(content=INTENT_SYSTEM_PROMPT),
        HumanMessage(content=user_question)
    ]
    response = intent_llm(messages)
    return response.content.strip().lower()


C:\Users\keong\AppData\Local\Temp\ipykernel_2021336\3671568503.py:29: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  intent_llm = ChatOpenAI(


In [3]:
gk_query = "Who invented the light bulb?"
si_query = "Where is your store located?"
db_query = "Is item #12345 available for purchase?"
unclear_query = "What about that thing?"
result1 = classify_intent(gk_query)
print(result1)
result2 = classify_intent(si_query)
print(result2)
result3 = classify_intent(db_query)
print(result3)
result4 = classify_intent(unclear_query)
print(result4)

general_knowledge
store_info
database_query
unclear


In [7]:
from src.shared.constant import *

import os
import pandas as pd
import yaml
import json
import re

def load_knowledge_base(file_path: str):
    """
    Load a knowledge base file and normalize it into
    texts + metadatas for vector database ingestion.

    Returns:
        dict:
            {
              "texts": List[str],
              "metadatas": List[dict]
            }
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Knowledge base file not found: {file_path}")

    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".txt":
        return _load_txt_schema(file_path)

    elif ext == ".csv":
        return _load_csv(file_path)

    elif ext in [".xls", ".xlsx"]:
        return _load_excel(file_path)

    elif ext == ".json":
        return _load_json(file_path)

    elif ext in [".yml", ".yaml"]:
        return _load_yaml(file_path)

    else:
        raise ValueError(
            f"Unsupported file type '{ext}'. Supported types: CSV, Excel, JSON, YAML, TXT."
        )

def _load_txt_schema(file_path: str):
    with open(file_path, "r", encoding="utf-8") as f:
        raw = f.read()

    # Split by separator lines
    blocks = re.split(r"=+\n", raw)

    texts = []
    metadatas = []

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        # Extract English table name
        table_match = re.search(r"Table:\s*(.+)", block, re.IGNORECASE)
        if not table_match:
            continue

        table_name = (
            table_match.group(1)
            .strip()
            .lower()
            .replace(" ", "_")
        )

        texts.append(block)
        metadatas.append({
            "type": "schema",
            "table": table_name,
            "source": "database",
            "file": os.path.basename(file_path)
        })

    return {
        "texts": texts,
        "metadatas": metadatas
    }

def _load_csv(file_path: str):
    df = pd.read_csv(file_path)

    texts = []
    metadatas = []

    for idx, row in df.iterrows():
        texts.append(row.to_string())
        metadatas.append({
            "type": "table_row",
            "row": idx,
            "file": os.path.basename(file_path)
        })

    return {
        "texts": texts,
        "metadatas": metadatas
    }


def _load_excel(file_path: str):
    df = pd.read_excel(file_path)

    texts = []
    metadatas = []

    for idx, row in df.iterrows():
        texts.append(row.to_string())
        metadatas.append({
            "type": "table_row",
            "row": idx,
            "file": os.path.basename(file_path)
        })

    return {
        "texts": texts,
        "metadatas": metadatas
    }

def _load_json(file_path: str):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    text = json.dumps(data, ensure_ascii=False, indent=2)

    return {
        "texts": [text],
        "metadatas": [{
            "type": "json",
            "file": os.path.basename(file_path)
        }]
    }


def _load_yaml(file_path: str):
    with open(file_path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)

    text = yaml.dump(data, allow_unicode=True)

    return {
        "texts": [text],
        "metadatas": [{
            "type": "yaml",
            "file": os.path.basename(file_path)
        }]
    }


In [10]:
import os
from langchain.embeddings import OpenAIEmbeddings

def get_openai_embedding_model():
    """
    Centralized embedding factory.
    Swappable in the future without refactoring.
    """
    return OpenAIEmbeddings(
        model="text-embedding-3-small",
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        request_timeout=30,
        max_retries=3
    )

from langchain.schema import Document

def build_documents(texts: list[str], metadatas: list[dict] | None = None):
    """
    Converts raw text + metadata into LangChain Documents.
    """
    if metadatas and len(texts) != len(metadatas):
        raise ValueError("texts and metadatas must have same length")

    documents = []
    for i, text in enumerate(texts):
        documents.append(
            Document(
                page_content=text.strip(),
                metadata=metadatas[i] if metadatas else {}
            )
        )
    return documents

from langchain.vectorstores import FAISS
import os
import shutil

def build_vector_database(
    documents: list,
    persist_path: str,
    rebuild: bool = False
):
    """
    Builds and persists a FAISS vector database using OpenAI embeddings.

    Args:
        documents: List[Document]
        persist_path: Directory path to save vector DB
        rebuild: If True, deletes existing DB and rebuilds

    Returns:
        FAISS vectorstore
    """

    if rebuild and os.path.exists(persist_path):
        shutil.rmtree(persist_path)

    embedding_model = get_openai_embedding_model()

    vectorstore = FAISS.from_documents(
        documents=documents,
        embedding=embedding_model
    )

    os.makedirs(persist_path, exist_ok=True)
    vectorstore.save_local(persist_path)

    return vectorstore

def load_vector_database(persist_path: str):
    """
    Loads a persisted FAISS vector database.
    """
    if not os.path.exists(persist_path):
        raise FileNotFoundError("Vector database not found")

    embedding_model = get_openai_embedding_model()

    return FAISS.load_local(
        persist_path,
        embedding_model,
        allow_dangerous_deserialization=True
    )

def retrieve_documents(
    vectorstore,
    query: str,
    k: int = 4,
    score: bool = False
):
    """
    Retrieves relevant documents for a query.

    Returns:
        List[Document] or List[(Document, score)]
    """
    if score:
        return vectorstore.similarity_search_with_score(query, k=k)
    return vectorstore.similarity_search(query, k=k)


In [ ]:
kb = load_knowledge_base(KNOWLEDGE_BASE_PATH)


documents = build_documents(
    texts=kb["texts"],
    metadatas=kb["metadatas"]
)

schema_vectorstore = build_vector_database(
    documents=documents,
    persist_path="vector_db/schema",
    rebuild=True
)


schema_vectorstore = load_vector_database("vector_db/schema")

docs = retrieve_documents(
    schema_vectorstore,
    query="How many gold rings sold last month?",
    k=10
)
